# Portfolio Simulator

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import seaborn as sns
from plotly import tools
import plotly.graph_objs as go
from lib import plot as p
from lib import data as d
from lib import finance as f

In [2]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

lpalette = {'g':'#13E881', 'r':'#FF5949', 'o':'#FFB84C', 'b':'#A7BEFA', 'w':'#FFFFFF', 'pk':'#FFA1FD', 'bg':'#FFEDBA', 'c':'#90EBF0',
            'pp':'#BBA0F0', 'g2':'#CCFFED', 'b2':'#E3EEFF', 'y':'#FFFC57'}
dpalette = {'g':'#268040', 'r':'#800B00', 'o':'#A13808', 'b':'#464CC2', 'w':'#B8BFBA', 'pk':'#A04EA6', 'bg':'#807155', 'c':'#1D4544',
            'pp':'#291147', 'g2':'#394742', 'b2':'#414449', 'y':'#666523'}

In [3]:
fig_asset = p.make_fig(cols=2, specs=[[{},{}]])
fig_asset_1 = p.make_fig(cols=2, specs=[[{},{}]])

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



#### Reading data

In [4]:
dquote = pd.read_csv('data/br_quotes.csv')
dstocks = pd.read_csv('data/br_stocks.csv')
dnu, dport, dport_inv, dprices = f.read_invest()

dflow = dnu.groupby('ID').apply(f.calc_flows)
dflow = dflow.reset_index(level=[0, 1])
dflow.pop('level_1')
dflow = dflow.sort_values(by=['Rendimento anual'], ascending=False).reset_index(drop=True)

#### Calculating stocks return

In [5]:
dreturn = pd.DataFrame()
current_year = dt.date.today().year
dreturn['Ticker'] = dquote['Ticker']
dreturn['Predicted'] = [0 for j in dquote['Ticker']]
weights = [0.05, 0.1, 0.15, 0.3, 0.4]
for i in range(5):
    year_i = current_year-4+i
    dreturn[str(year_i)] = (dquote[str(year_i)] - dquote[str(year_i-1)]) / dquote[str(year_i-1)] 
    dreturn[str(year_i)] = dreturn[str(year_i)].apply(lambda x : 0 if pd.isnull(x) or x == np.inf else x)
    dreturn['Predicted'] += dreturn[str(year_i)] * weights[i]

# Screener

In [6]:
assets = ['USA', 'EU', 'BR', 'EM', 'Bonds', 'Cash', 'Commod']
industry = ['Basic Mat', 'Cons Cycl', 'Finance', 'Real Estate', 'Telecom', 'Energy', 'Indust', 'Tech', 'Cons Non-Cycl', 'Health', 'Util']

dfull = f.calc_stocks_feat(dreturn, dstocks, dquote, assets, industry)

screen = ['Ticker', 'Score', 'Price', 'Return', 'Risk', 'Tax', 'YTD', 'Cat', 'Geo', 'Domain', 'Asset']
p.change_df_prop(dfull[screen].sort_values('Score', ascending=False).reset_index(drop=True))

Ticker,Score,Price,Return,Risk,Tax,YTD,Cat,Geo,Domain,Asset
IVVB11,6.40991,139.4,15.12%,12.18%,0.24%,0.5%,2,USA,Tech,Equity
Cash,6.10837,6.7,0.0%,0.0%,0.0%,0.0%,0,Cash,Basic Mat,Cash
DIVO11,4.28036,48.5,8.57%,19.25%,0.5%,-30.22%,3,USA,Util,Equity
BBSD11,4.26023,64.38,7.24%,17.73%,0.5%,-32.75%,3,USA,Indust,Equity
BRAX11,3.61888,61.85,2.14%,17.38%,0.2%,-35.47%,1,BR,Finance,Equity
BOVB11,3.35227,73.48,0.99%,17.73%,0.2%,-36.49%,1,BR,Finance,Equity
BOVV11,3.3247,73.7,1.13%,17.73%,0.3%,-36.6%,1,BR,Finance,Equity
PIBB11,3.18598,122.8,0.08%,17.68%,0.06%,-37.07%,1.1,BR,Finance,Equity
GOVE11,2.93034,32.66,3.31%,22.55%,0.5%,-35.2%,1,BR,Finance,Equity
MATB11,2.33879,24.53,-0.33%,22.55%,0.5%,-32.16%,4,BR,Basic Mat,Equity


# Current portfolio

In [7]:
dport_bonds = f.simulatePortfolio(dport, dflow, dfull, assets, industry)

In [8]:
dport_bonds[['Ticker', 'Buy', 'Shares', 'Date']]

,Ticker,Buy,Shares,Date
0,IVVB11,130.800,10,2019-10-30
1,BRAX11,87.500,10,2019-11-14
2,IVVB11,139.030,10,2019-12-12
3,BBSD11,95.720,10,2020-01-09
4,BRAX11,97.030,10,2020-01-16
5,IVVB11,147.800,10,2020-01-31
6,BBSD11,96.670,10,2020-02-07
7,BRAX11,96.400,10,2020-02-21
8,BBSD11,89.420,10,2020-03-04
9,Cash,6.700,1,2020-02-01


### Portfolio performance (w/ bonds)

In [9]:
dcurrent_bonds = f.calc_port_perfo(dport_bonds)
p.change_df_prop(dcurrent_bonds[['Value', 'Profit', 'Yield', 'Return', 'Risk']], 28)

Value,Profit,Yield,Return,Risk
32319.9,-1561.14,-4.83%,4.77%,3.63%


### Portfolio performance

In [10]:
dport_var = dport_bonds[dport_bonds['Ticker'] != 'NU']
dport_var = dport_var[dport_var['Ticker'] != 'Cash']
dcurrent_var = f.calc_port_perfo(dport_var)
p.change_df_prop(dcurrent_var[['Value', 'Profit', 'Yield', 'Return', 'Risk']], 28)

Value,Profit,Yield,Return,Risk
7968.9,-1834.8,-23.02%,10.19%,14.74%


### Asset allocation

In [11]:
palette = {'BR': lpalette['g'], 'Cash': lpalette['c'], 'Bonds': dpalette['b'], 'EU': lpalette['pp'], 'USA': lpalette['r'], 'EM':lpalette['pk'],
           'Finance': lpalette['g'], 'Indust': lpalette['c'], 'Tech': dpalette['b'], 'Cons Non-Cycl': lpalette['pp'], 'Cons Cycl': lpalette['r'], 'Health':lpalette['pk'],
           'Energy': dpalette['g'], 'Util': dpalette['c'], 'Basic Mat': dpalette['o'], 'Real Estate': dpalette['pp'], 'Telecom': dpalette['r'], 'Commod':lpalette['pk'],
           'BRAX11': lpalette['g'], 'Cash': lpalette['c'], 'NU': dpalette['b'], 'BBSD11': lpalette['pp'], 'IVVB11': lpalette['r'], 'Portfolio': lpalette['b2']
          }
dport_bonds['Group'] = 1
dgroup = d.pivot_agg(dport_bonds, val='Value', idx=['Group'], col=['Ticker'], agg={'Value':np.sum}, date=None)
dgroup.pop('Group')
dgroup = dgroup.sum()
# dgroup.pop('Ticker')
p.plot_pie(dgroup, fig_asset, palette, dict(x=[0, 0.33]))
dgroup = dport_bonds.sum()
p.plot_pie(dgroup[assets], fig_asset, palette, dict(x=[0.33, 0.66]))
p.plot_pie(dgroup[industry], fig_asset, palette, dict(x=[0.66, 1]))

FigureWidget({
    'data': [{'domain': {'x': [0, 0.33]},
              'hole': 0.4,
              'labels': ar…

# Portfolio 1

In [12]:
dport_1 = f.addStocks(dport, [
        ['IVVB11', 80, 0.6],
        ['BRAX11', 70, 0.4],
        ['BBSD11', 70, 0.3],  
], dfull)

In [13]:
dport_bonds = f.simulatePortfolio(dport_1, dflow, dfull, assets, industry, bonds=11000).reset_index(drop=True)

In [14]:
dport_bonds[['Ticker', 'Buy', 'Shares']]

,Ticker,Buy,Shares
0,IVVB11,130.800,10
1,BRAX11,87.500,10
2,IVVB11,139.030,10
3,BBSD11,95.720,10
4,BRAX11,97.030,10
5,IVVB11,147.800,10
6,BBSD11,96.670,10
7,BRAX11,96.400,10
8,BBSD11,89.420,10
9,Cash,6.700,1


### Portfolio performance (w/ bonds)

In [15]:
dcurrent_bonds_1 = f.calc_port_perfo(dport_bonds)
p.change_df_prop(dcurrent_bonds_1[['Value', 'Profit', 'Yield', 'Return', 'Risk']], 28)

Value,Profit,Yield,Return,Risk
38963.7,-1561.14,-4.01%,8.34%,10.48%


### Portfolio performance

In [16]:
dport_var = dport_bonds[dport_bonds['Ticker'] != 'NU']
dport_var = dport_var[dport_var['Ticker'] != 'Cash']
dcurrent_var_1 = f.calc_port_perfo(dport_var)
p.change_df_prop(dcurrent_var_1[['Value', 'Profit', 'Yield', 'Return', 'Risk']], 28)

Value,Profit,Yield,Return,Risk
27957,-1834.8,-6.56%,10.44%,14.61%


### Asset allocation

In [17]:
palette = {'BR': lpalette['g'], 'Cash': lpalette['c'], 'Bonds': dpalette['b'], 'EU': lpalette['pp'], 'USA': lpalette['r'], 'EM':lpalette['pk'],
           'Finance': lpalette['g'], 'Indust': lpalette['c'], 'Tech': dpalette['b'], 'Cons Non-Cycl': lpalette['pp'], 'Cons Cycl': lpalette['r'], 'Health':lpalette['pk'],
           'Energy': dpalette['g'], 'Util': dpalette['c'], 'Basic Mat': dpalette['o'], 'Real Estate': dpalette['pp'], 'Telecom': dpalette['r'], 'Commod':lpalette['pk'],
           'BRAX11': lpalette['g'], 'Cash': lpalette['c'], 'NU': dpalette['b'], 'BBSD11': lpalette['pp'], 'IVVB11': lpalette['r'], 'Portfolio': lpalette['b2']
          }
dport_bonds['Group'] = 1
dgroup = d.pivot_agg(dport_bonds, val='Value', idx=['Group'], col=['Ticker'], agg={'Value':np.sum}, date=None)
dgroup.pop('Group')
dgroup = dgroup.sum()
# dgroup.pop('Ticker')
p.plot_pie(dgroup, fig_asset_1, palette, dict(x=[0, 0.33]))
dgroup = dport_bonds.sum()
p.plot_pie(dgroup[assets], fig_asset_1, palette, dict(x=[0.33, 0.66]))
p.plot_pie(dgroup[industry], fig_asset_1, palette, dict(x=[0.66, 1]))

FigureWidget({
    'data': [{'domain': {'x': [0, 0.33]},
              'hole': 0.4,
              'labels': ar…

# Portfolio comparison

In [18]:
dcompare = dcurrent_bonds.append(dcurrent_bonds_1).reset_index(drop=True)
p.change_df_prop(dcompare[['Value', 'Profit', 'Yield', 'Return', 'Risk']], 28)

Value,Profit,Yield,Return,Risk
32319.9,-1561.14,-4.83%,4.77%,3.63%
38963.7,-1561.14,-4.01%,8.34%,10.48%
